# Lab Exercise
## De-Duping Data

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL on a file we've received from some customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraincentralindia.blob.core.windows.net/

Created user-specific database

Using the database krishthan2109_outlook_com_db .

All done!

In [0]:
%run "./Includes/Initialize-Labs"

Initializing lab environment: 
 Declared clearYourResults( passedOnly:Boolean=true ) 
 Declared validateYourSchema( what:String, df:DataFrame, expColumnName:String, expColumnType:String ) 
 Declared validateYourAnswer( what:String, expectedHash:Int, answer:Any ) 
 Declared summarizeYourResults() 
 Declared logYourTest( path:String, name:String, value:Double ) 
 Declared loadYourTestResults( path:String ) returns DataFrame 
 Declared loadYourTestMap( path:String ) returns Map[String,Double]

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4245474461494926> in <module> 
 ----> 1 dbutils . fs . head ( ) 
 2 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 371 def f_with_exception_handling ( * args , ** kwargs ) : 
 372 try : 
 --> 373 return f ( * args , ** kwargs ) 
 374 except Py4JJavaError as e : 
 375 

 TypeError : head() missing 1 required positional argument: 'file'

In [0]:
dbutils.fs.ls


Out[40]: <bound method DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling of Package 'dbutils.fs'. For more information, type 'display(dbutils.fs)' in a cell.>

In [0]:
# TODO

(source, sasEntity, sasToken) = getAzureDataSource()
spark.conf.set(sasEntity, sasToken)

sourceFile = source + "/dataframes/people-with-dups.txt"
destFile = userhome + "/people.parquet"

# In case it already exists
dbutils.fs.rm(destFile, True)

Out[38]: False

In [0]:
display(dbutils.fs.ls(userhome))

path,name,size,modificationTime
dbfs:/user/krishthan2109@outlook.com/activity/,activity/,0,1662443671000


In [0]:
DF1 = spark.read.option("inferSchema","true").option("header","true").option("delimiter",":").csv(sourceFile).cache()

DF1.count()


Out[100]: 103000

In [0]:
DF1.show(5)

+---------+----------+---------+------+-------------------+------+-----------+
firstName|middleName| lastName|gender| birthDate|salary| ssn|
+---------+----------+---------+------+-------------------+------+-----------+
 Emanuel| Wallace| Panton| M|1988-03-04 00:00:00|101255|935-90-7627|
 Eloisa| Rubye|Cayouette| F|2000-06-20 00:00:00|204031|935-89-9009|
 Cathi| Svetlana| Prins| F|2012-12-22 00:00:00| 35895|959-30-7957|
 Mitchel| Andres|Mozdzierz| M|1966-05-06 00:00:00| 55108|989-27-8093|
 Angla| Melba|Hartzheim| F|1938-07-26 00:00:00| 13199|935-27-4276|
+---------+----------+---------+------+-------------------+------+-----------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
DF2 = (DF1.withColumn(("firstName"),lower("firstName"))
      .withColumn(("middleName"),lower("middleName"))
      .withColumn(("lastName"),lower("lastName"))
      .withColumn(("gender"),lower("gender"))
      .withColumn(("ssn"),translate(col("ssn"), "-", "")))

DF3 = DF2.drop_duplicates()
DF3.count()


Out[95]: 100000

In [0]:
DF3.write.mode("overwrite").parquet(destFile)

##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()


01 Expected 100000 Records was correct, your answer: 100000

01 Expected 100000 Records:,passed,100000
